In [3]:
import pandas as pd

data=pd.read_excel("/home/sergio/Desktop/Courses/Prueba Técnica/database_clean (3) (1).xlsx")

In [7]:
import torch
from transformers import AutoModelForCausalLM , AutoTokenizer

class LMHeadModel:

    def __init__(self, model_name):
        # Initialize the model and the tokenizer.
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    def get_predictions(self, sentence):
        # Encode the sentence using the tokenizer and return the model predictions.
        inputs = self.tokenizer.encode(sentence, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(inputs)
            predictions = outputs[0]
        return predictions
    
    def get_next_word_probabilities(self, sentence, top_k=500):

        # Get the model predictions for the sentence.
        predictions = self.get_predictions(sentence)
        
        # Get the next token candidates.
        next_token_candidates_tensor = predictions[0, -1, :]

        # Get the top k next token candidates.
        topk_candidates_indexes = torch.topk(
            next_token_candidates_tensor, top_k).indices.tolist()

        # Get the token probabilities for all candidates.
        all_candidates_probabilities = torch.nn.functional.softmax(
            next_token_candidates_tensor, dim=-1)
        
        # Filter the token probabilities for the top k candidates.
        topk_candidates_probabilities = \
            all_candidates_probabilities[topk_candidates_indexes].tolist()

        # Decode the top k candidates back to words.
        topk_candidates_tokens = \
            [self.tokenizer.decode([idx]).strip() for idx in topk_candidates_indexes]

        # Return the top k candidates and their probabilities.
        return list(zip(topk_candidates_tokens, topk_candidates_probabilities))


sentence = "Creame un cuento"
model = LMHeadModel("PlanTL-GOB-ES/gpt2-large-bne")
model.get_next_word_probabilities(sentence, top_k=500)

[('de', 0.184383362531662),
 (',', 0.1210019662976265),
 ('y', 0.06817547976970673),
 ('para', 0.04182649403810501),
 ('que', 0.038427818566560745),
 ('con', 0.03270898759365082),
 ('en', 0.025280438363552094),
 (':', 0.02337273582816124),
 ('sobre', 0.02302161417901516),
 ('a', 0.022492630407214165),
 ('...', 0.022490421310067177),
 ('.', 0.02050693891942501),
 ('o', 0.016299769282341003),
 ('al', 0.012165495194494724),
 ('es', 0.008341645821928978),
 ('(', 0.007145529147237539),
 ('por', 0.006954090669751167),
 ('antes', 0.006322361528873444),
 ('chino', 0.005846717860549688),
 ('muy', 0.005716733634471893),
 ('donde', 0.004779293667525053),
 ('un', 0.004435393493622541),
 ('si', 0.004378952085971832),
 ('infantil', 0.003933524247258902),
 ('del', 0.0036908879410475492),
 ('!', 0.0031855395063757896),
 ('no', 0.003063848242163658),
 (';', 0.0028071498963981867),
 ('más', 0.002722724573686719),
 ('d', 0.002654953161254525),
 ('nuevo', 0.002606213791295886),
 ('....', 0.002580560278147

In [12]:
sentence = "no sé si me pasa a mí, pero la copa libertadores, por alguna razón, no genera la expectativa de antes. por ahí puede haber un buen juego, pero la gente se entera"
a=model.get_next_word_probabilities(sentence, top_k=500)

[('de', 0.42379918694496155),
 ('y', 0.06279395520687103),
 ('que', 0.05345726013183594),
 ('por', 0.05261974781751633),
 (',', 0.041026052087545395),
 ('.', 0.028506837785243988),
 ('más', 0.02844405360519886),
 ('del', 0.023334065452218056),
 ('cuando', 0.019266391173005104),
 ('en', 0.01854119449853897),
 ('antes', 0.015516676008701324),
 ('a', 0.013777803629636765),
 ('muy', 0.012518671341240406),
 ('un', 0.010024932213127613),
 ('menos', 0.008465590886771679),
 ('demasiado', 0.008404486812651157),
 ('poco', 0.008284837938845158),
 ('porque', 0.008200843818485737),
 ('mucho', 0.007984261028468609),
 ('ahora', 0.007966280914843082),
 ('(', 0.007542294450104237),
 ('al', 0.00736639741808176),
 ('o', 0.007042364217340946),
 ('enseguida', 0.006591499783098698),
 ('tarde', 0.006253169383853674),
 ('solo', 0.006188480649143457),
 ('sólo', 0.005644205957651138),
 ('con', 0.005496406462043524),
 ('ya', 0.004921391140669584),
 ('después', 0.004501013550907373),
 ('rápido', 0.003306196536868

: 